In [1]:
import os
os.chdir("..")
import pandas as pd
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import Counter
import json
root_dir = "data/refit"
datetime_temp = "%Y-%m-%d %H:%M:%S"
pd.set_option('display.max_rows', None)
def avg_duration_evt(data_x, data_y, onthreshold, min_on = timedelta(minutes=1)):
    pre = data_x[0]
    state = "off"
    on = []
    for i,d in enumerate(data_y):
        if i > 0 and data_x[i] - data_x[i-1] > timedelta(hours=1):
            if state == "on":
                state = "off"
                this_on = data_x[i] - pre
                if this_on < min_on:
                    continue
                on.append(this_on.seconds//60)
                if data_x[i-1] - pre > timedelta(hours=10):
                    print(pre)

        if d >= onthreshold and state == "off":
            state = "on"
            pre = data_x[i]
        if d < onthreshold and state == "on":
            state = "off"
            this_on = data_x[i] - pre
            if this_on < min_on:
                continue
            on.append(this_on.seconds//60)
            if on[-1] == 1:
                print(pre)
            if data_x[i] - pre > timedelta(hours=10):
                print(pre)
    print(min(on))
    print(max(on))
    return sum(on) / (len(on) + 0.000001), on   

def get_freq_cols(df):
    cols = list(df.columns)
    freq = {}
    for c in cols:
        if "Appliance" not in c:
            continue
        freq[c] = df[c].value_counts()
    return freq

In [ ]:

filename = "House1.csv"
file_path = os.path.join(root_dir, filename)
meter = pd.read_csv(file_path)
freq_cnt = get_freq_cols(meter)

# y = meter["Appliance7"]
# x = range(len(y))
time = [datetime.strptime(x,datetime_temp) for x in meter["Time"]]
last = 40000
# print(avg_duration_evt(time, y, 0.001))
# plt.ylim(0,0.3)


In [ ]:
filename = "Weather.csv"
file_path = os.path.join(root_dir, filename)
weather = pd.read_csv(file_path)
a = set([x.split("'")[-2] for x in weather["weatherDesc"]])

time = datetime.strptime(weather["date_time"][0], datetime_temp)
print(min(weather["humidity"]), max(weather["humidity"]))



In [ ]:
# print(freq_cnt["Appliance9"])
y = meter["Appliance9"]
avg, on = avg_duration_evt(time, y, 500, min_on = timedelta(minutes=5))
print("The number of on event " + str(len(on)))
print("Average on time " + str(avg))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=y,mode='markers', name='markers'))
fig.show()

In [2]:
from RefitProcessor import RefitProcessor
from config import *
from datetime import timedelta
logging.basicConfig(level=PROJ_LOGGING_LEVEL)

def prepare_refit_data():
    test_project = "refit"
    project_path = os.path.join("../" + DATA_ROOT, test_project)
    device_list = {"House1.csv": [
                        {"name": "Appliance7", 
                        "onThreshold": 20, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "PC"},
                        {"name": "Appliance5", 
                        "onThreshold": 0, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "WashingMachine"},
                        {"name": "Appliance8", 
                        "onThreshold": 20, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
                    "House3.csv": [
                        {"name": "Appliance6", 
                        "onThreshold": 0, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "WashingMachine"},
                        {"name": "Appliance7", 
                        "onThreshold": 20, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
                    "House4.csv": [
                        {"name": "Appliance5", 
                        "onThreshold": 9, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "PC"},
                        {"name": "Appliance4", 
                        "onThreshold": 5, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "WashingMachine"},
                        {"name": "Appliance7", 
                        "onThreshold": 30, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
                    "House8.csv": [
                        {"name": "Appliance6", 
                        "onThreshold": 60, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "PC"},
                        {"name": "Appliance4", 
                        "onThreshold": 5, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "WashingMachine"},
                        {"name": "Appliance7", 
                        "onThreshold": 30, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
                    "House9.csv": [
                        {"name": "Appliance3", 
                        "onThreshold": 1, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "WashingMachine"},
                        {"name": "Appliance5", 
                        "onThreshold": 60, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
                    "House15.csv": [
                        {"name": "Appliance6", 
                        "onThreshold": 89, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "PC"},
                        {"name": "Appliance3", 
                        "onThreshold": 0, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "WashingMachine"},
                        {"name": "Appliance6", 
                        "onThreshold": 60, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
                    "House18.csv": [
                        {"name": "Appliance7", 
                        "onThreshold": 50, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "PC"},
                        {"name": "Appliance5", 
                        "onThreshold": 0, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "WashingMachine"},
                        {"name": "Appliance8", 
                        "onThreshold": 50, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
                    "House20.csv": [
                        {"name": "Appliance6", 
                        "onThreshold": 70, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "PC"},
                        {"name": "Appliance4", 
                        "onThreshold": 0, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "WashingMachine"},
                        {"name": "Appliance7", 
                        "onThreshold": 50, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
    }
    context_list = {
        "Weather.csv": [
            {"name": "FeelsLikeC"},
            {"name": "cloudcover"},
            {"name": "humidity"},
            {"name": "weatherDesc", "lambda": lambda x: x.split("'")[-2]},
            {"name": "tempC"},
        ]
    }

    refit_processor = RefitProcessor(project_path, context_list, device_list)
    # ctx_evts, device_evts = refit_processor.preprocess(output_file="p")

prepare_refit_data()